##Alunos:
*   Fabio Cardoso - fc2@cesar.school
*   Andréa Fonseca - asf4@cesar.school

##Dataset
https://www.kaggle.com/datasets/juniorbueno/neural-networks-homer-and-bart-classification

## Importação das bibliotecas

In [1]:
from os.path import exists
import torch
import zipfile

In [2]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

##Importando o dataset

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
path = '/content/gdrive/My Drive/Bart_Homer.zip'
zip_object = zipfile.ZipFile(file = path, mode = 'r')
zip_object.extractall('./')
zip_object.close()

FileNotFoundError: [Errno 2] No such file or directory: 'https://drive.google.com/file/d/1b1b1Zay2_TiNd2XvEdKOjaucQgLVryjb/view?usp=drive_link'

In [ ]:
data_dir_train = '/content/Bart_Homer/Treinamento'
data_dir_test = '/content/Bart_Homer/Teste'

##Definição da Rede Neural

In [ ]:
class NetBartHomer(nn.Module):
    def __init__(self):
        super(NetBartHomer, self).__init__()
        self.dense = nn.Sequential(
            nn.Linear(224 * 224 * 3, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 2)
        )

    def forward(self, x):
      x = x.view(-1, 224 * 224 * 3) # Reshape to match the expected input size
      x = self.dense(x)
      output = F.log_softmax(x, dim=1)
      return output

model = NetBartHomer()

##Função para treinamento do modelo

In [ ]:
def train(log_interval, dry_run, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target[:output.size(0)])
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if dry_run:
                break

##Função para o teste do modelo

In [ ]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

##Função do modelo e avaliação

In [ ]:
use_cuda = torch.cuda.is_available()

torch.manual_seed(14)

device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': 5}
test_kwargs = {'batch_size': 25}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                    'pin_memory': True,
                    'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform_train=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=7, translate=(0, 0.07), shear=0.2, scale=(1, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

transform_test = transforms.Compose(
    [
     transforms.Resize([224,224]),
     transforms.ToTensor()
    ]
)

dataset1 = train_dataset = datasets.ImageFolder(data_dir_train, transform=transform_train)
dataset2 = test_dataset = datasets.ImageFolder(data_dir_test, transform=transform_test)

train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = NetBartHomer().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=0.1)

epochs = 20
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

for epoch in range(1, epochs + 1):
    train(10, False, model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

torch.save(model.state_dict(), "homer_bart.pt")

Train Epoch: 1 [0/202 (0%)]	Loss: 0.697115
Train Epoch: 1 [50/202 (24%)]	Loss: 0.024825
Train Epoch: 1 [100/202 (49%)]	Loss: 1.891869
Train Epoch: 1 [150/202 (73%)]	Loss: 1.543227
Train Epoch: 1 [80/202 (98%)]	Loss: 1.522758

Test set: Average loss: 0.8405, Accuracy: 40/67 (60%)

Train Epoch: 2 [0/202 (0%)]	Loss: 1.723411
Train Epoch: 2 [50/202 (24%)]	Loss: 0.661287
Train Epoch: 2 [100/202 (49%)]	Loss: 0.449178
Train Epoch: 2 [150/202 (73%)]	Loss: 0.656151
Train Epoch: 2 [80/202 (98%)]	Loss: 0.316702

Test set: Average loss: 0.6605, Accuracy: 40/67 (60%)

Train Epoch: 3 [0/202 (0%)]	Loss: 0.149667
Train Epoch: 3 [50/202 (24%)]	Loss: 0.583143
Train Epoch: 3 [100/202 (49%)]	Loss: 0.713685
Train Epoch: 3 [150/202 (73%)]	Loss: 0.696687
Train Epoch: 3 [80/202 (98%)]	Loss: 0.156514

Test set: Average loss: 0.6518, Accuracy: 40/67 (60%)

Train Epoch: 4 [0/202 (0%)]	Loss: 0.831799
Train Epoch: 4 [50/202 (24%)]	Loss: 0.619585
Train Epoch: 4 [100/202 (49%)]	Loss: 0.563846
Train Epoch: 4 [150/202